In [1]:
# remove folders from previous executions of this notebook first, otherwise it won't work
try
    rm("MyTest", recursive=true)
catch e 
    println("MyTest already deleted.")
end 

try 
    rm("MyTestTwo", recursive=true)
catch e 
    println("MyTestTwo already deleted.")
end

# Packages and Modules (Part II) 

* We talked about Julia's `Pkg` and `module` before, today we will see how you write your own packages, so that you can do that to do a simple SIR package
* Note: In this notebook I sometimes write the input of a cell into a file, when you execute this notebook on your computer, you have to make sure that the cell number is correct, you might have to change it. But usually you will much rather code modules in editors/IDEs like VSCode(ium). 


## Pkg

Hopefully, you remember the basic usage of `Pkg` Julia's system that manages enviroments and packages

* `activate` actvites an environment 
* `add`, `rm` add and remove packages from the current environment 
* `up` updates the current environment 
* `status` displays information about the current environment
* `test` tests if a package works correctly on our system

The dependencies and the status of an environment are stored in the `Manifest.toml` and `Project.toml` files that are automatically created and usually also automatically edited.

We saw that we can generate a new project like this:

In [2]:
]generate MyTest

  Generating  project MyTest:
    MyTest/Project.toml
    MyTest/src/MyTest.jl


In [3]:
;tree MyTest

MyTest
├── Project.toml
└── src
    └── MyTest.jl

1 directory, 2 files


In [4]:
;less MyTest/src/MyTest.jl

module MyTest

greet() = print("Hello World!")

end # module


The `generate` command already gives us the basic structure of a package with a `Project.toml` for the environment and the `MyTest.jl` inside the `src` folder that defines the `module` `MyTest`. We can do this also slightly more advanced with `PkgTemplates`. 

### PkgTemplates

* `PkgTemplates` provides a more sophistaced initialization that can e.g. include GitHub, CI/CD, Licences, etc. 
* It works by defining a template
* The minimum is: 
    * The GitHub or Gitlab username `user`
    * Your name and mail `authors`
    * The directory on your local machine `dir` 
    * The minimum required Julia version `julia`

In [5]:
using PkgTemplates
tmp = Template(user="my-github-user-name",authors="My Name <mail@adress.com>",dir=".",julia=v"1.6.0",plugins=[Documenter()])
tmp("MyTestTwo.jl")

┌ Info: Running prehooks
└ @ PkgTemplates /Users/max/.julia/packages/PkgTemplates/j6Nfl/src/template.jl:130
┌ Info: Running hooks
└ @ PkgTemplates /Users/max/.julia/packages/PkgTemplates/j6Nfl/src/template.jl:130
  Activating project at `~/Nextcloud/TUM-Dynamics-Lecture/lectures/lecture-7/MyTestTwo`
    Updating registry at `~/.julia/registries/General.toml`
┌ Warning: The active manifest file at `/Users/max/Nextcloud/TUM-Dynamics-Lecture/lectures/lecture-7/MyTestTwo/Manifest.toml` has an old format that is being maintained.
│ To update to the new format run `Pkg.upgrade_manifest()` which will upgrade the format without re-resolving.
└ @ Pkg.Types /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.7/Pkg/src/manifest.jl:287
  No Changes to `~/Nextcloud/TUM-Dynamics-Lecture/lectures/lecture-7/MyTestTwo/Project.toml`
  No Changes to `~/Nextcloud/TUM-Dynamics-Lecture/lectures/lecture-7/MyTestTwo/Manifest.toml`
Precompiling project...
  ✓ MyTestTwo
  1 dependency s

It's handy to save the `Template` call somewhere, e.g. directly in Julia's startup script, see [PkgTemplates doc for different options](https://juliaci.github.io/PkgTemplates.jl/stable/user/#Saving-Templates-1).

This creates the basic package structure: 

In [6]:
;tree MyTestTwo

MyTestTwo
├── LICENSE
├── Manifest.toml
├── Project.toml
├── README.md
├── docs
│   ├── Manifest.toml
│   ├── Project.toml
│   ├── make.jl
│   └── src
│       └── index.md
├── src
│   └── MyTestTwo.jl
└── test
    └── runtests.jl

4 directories, 10 files


In [7]:
;cd MyTestTwo

/Users/max/Nextcloud/TUM-Dynamics-Lecture/lectures/lecture-7/MyTestTwo


In [8]:
]activate .

  Activating project at `~/Nextcloud/TUM-Dynamics-Lecture/lectures/lecture-7/MyTestTwo`


In [9]:
]status

     Project MyTestTwo v0.1.0
      Status `~/Nextcloud/TUM-Dynamics-Lecture/lectures/lecture-7/MyTestTwo/Project.toml` (empty project)


This already initialised a few more files. It includes a license file 

In [10]:
;less LICENSE

MIT License

Copyright (c) 2022 My Name <mail@adress.com>

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF

An almost empty readme file:

In [11]:
;less README.md

# MyTestTwo


And the folder structure for tests which we will talk about in just a few minutes. Additionally, `PkgTemplates` provides additional, more advanced features via the keyword `plugins`. 
Plugins can modifiy some of these files and add further template files 

* `License(name="MIT", destination="LICENSE")` for the license file 
* `Git(....)` for the git configuration
* Options for CI/CD (we will talk about that in the next lectures)
* Further Options for tests
* `Documenter()` Documentation (can also include CI presets)
* Badges that are displayed on the GitHub page that link to the documentation, CI report, etc...
* [Full documentation of PackageTemplates](https://invenia.github.io/PkgTemplates.jl/dev/user/) for all further plugins that can be added 

## Modules 

* We first write a function for our module: 

In [12]:
"""
    add_two(x::T) where {T<:Number} 

Adds two to the input number.
"""
function add_two(x::T) where {T<:Number}
    x + T(2) 
end

"""
    add_two(x::AbstractArray{T,N}) where {T,N} 

Adds two to an array element-wise.
"""
function add_two(x::AbstractArray{T,N}) where {T,N}
    x .+ T(2)
end

add_two

For the documentation that we add later, we should include docstrings. There's no clear convention wheather you should write one docstring for every function of the same name or just one for all. The docstring should cover all the behaviour of all functions though!

We save that as a file in our package:

In [13]:
write("src/more_functions.jl",In[12])

297

* We've seen the `module` command before, it is the common practice that every package defines the module in the central `src/MyTestTwo.jl` file. This will usually look something like this: 

In [14]:
module MyTestTwo 
    using StatsBase # import other packages that your whole package needs 

    include("more_functions.jl") # include source code files where the actual functions of your project are 
    
    export add_two # export some of the functions that the users can use directly

    function __init__() # OPTIONAL: this special function is always executed when the module is loaded 
        nothing 
    end 
end 

LoadError: SystemError: opening file "/Users/max/Nextcloud/TUM-Dynamics-Lecture/lectures/lecture-7/MyTestTwo/more_functions.jl": No such file or directory

The error only occurs here in the Jupyter notebook, it will not occur when the package/module is called in the proper way

In [15]:
write("src/MyTestTwo.jl",In[14])

425

* `include`: The order of the files that you include matters, they are loaded sequentially, one after the other. If you e.g. define abstract types, you should do this in the first file you include (or sometimes also directly in the module file)
* `export`: All functions that are exported can be called directly after the user imports the library with `using MyTestTwo`, all other functions can be still called directly with e.g. `MyTestTwo.other_function`
* `__init__`: The init funcition is usually not necessary. However, it is sometimes used to determine what hardware is available and chooses the right functions based on that (e.g. whether a GPU is available or not)

In [16]:
;tree .

.
├── LICENSE
├── Manifest.toml
├── Project.toml
├── README.md
├── docs
│   ├── Manifest.toml
│   ├── Project.toml
│   ├── make.jl
│   └── src
│       └── index.md
├── src
│   ├── MyTestTwo.jl
│   └── more_functions.jl
└── test
    └── runtests.jl

4 directories, 11 files


In [17]:
]add StatsBase 

   Resolving package versions...
    Updating `~/Nextcloud/TUM-Dynamics-Lecture/lectures/lecture-7/MyTestTwo/Project.toml`
  [2913bbd2] + StatsBase v0.33.21
    Updating `~/Nextcloud/TUM-Dynamics-Lecture/lectures/lecture-7/MyTestTwo/Manifest.toml`
  [d360d2e6] + ChainRulesCore v1.15.6
  [9e997f8a] + ChangesOfVariables v0.1.4
  [34da2185] + Compat v4.5.0
  [9a962f9c] + DataAPI v1.13.0
  [864edb3b] + DataStructures v0.18.13
  [ffbed154] + DocStringExtensions v0.9.2
  [3587e190] + InverseFunctions v0.1.8
  [92d709cd] + IrrationalConstants v0.1.1
  [2ab3a3ac] + LogExpFunctions v0.3.19
  [e1d29d7a] + Missings v1.0.2
  [bac558e1] + OrderedCollections v1.4.1
  [a2af1166] + SortingAlgorithms v1.1.0
  [82ae8749] + StatsAPI v1.5.0
  [2913bbd2] + StatsBase v0.33.21
  [56f22d72] + Artifacts
  [2a0f44e3] + Base64
  [ade2ca70] + Dates
  [b77e0a4c] + InteractiveUtils
  [76f85450] + LibGit2
  [8f399da3] + Libdl
  [37e2e46d] + LinearAlgebra
  [56ddb016] + Logging
  [d6f4376e] + Markdown
  [ca575930] + 

We add all packages we imported to environment.

### Registry 

The Julia community manages all packages in regestries. 

In [18]:
]registry st

Registry Status 
 [23338594] General


* Normally, that is just the `General` registry that is itself a GitHub repository where all packages are listed  [General Registry](https://github.com/JuliaRegistries/General). 

* This repository is not managed manually but by bots. As long as a package follows the regular Julia package structure, it is relativly easy to add it to this registry with the [Registrator](https://github.com/JuliaRegistries/Registrator.jl/#registrator) bot.

## Tests

It is good practice to write unit tests for most functions of every package, that test if the package works as intended. They are particular practical to catch unwanted errors: You might make changes to one function that is also used by other functions and affects them. A properly written test makes sure that all functions still work as intended. 

Julia uses the `Test` package for that. In the package structure, all tests are written in the `test` subfolder that was created by `PkgTemplates` for us. These test are executed every time we call `]test MyTestTwo`. 

* Usually each important function is tested seperately 
* One should avoid large computation in tests, i.e. tests should run fairly quickly (~ minutes)
* The `runtests.jl` files is meant to load common packages and include all different tests

In [19]:
]add Test

   Resolving package versions...
    Updating `~/Nextcloud/TUM-Dynamics-Lecture/lectures/lecture-7/MyTestTwo/Project.toml`
  [8dfed614] + Test
  No Changes to `~/Nextcloud/TUM-Dynamics-Lecture/lectures/lecture-7/MyTestTwo/Manifest.toml`


In [20]:
using Test 

In [21]:
@test add_two(2.) ≈ 4.
    
@test add_two([2.,4.]) ≈ [4.,6.]

Test Passed
  Expression: add_two([2.0, 4.0]) ≈ [4.0, 6.0]
   Evaluated: [4.0, 6.0] ≈ [4.0, 6.0]

In [22]:
write("test/function_tests.jl",In[21])

64

In [23]:
using Test, MyTestTwo # import packages that all tests need 

# maybe you load some data or define some constant that all tests need 

@testset "Test MyTestTwo.jl Basics" begin
    include("function_tests.jl") # include individual tests 
end 

┌ Info: Precompiling MyTestTwo [98ba6296-b0dd-4d69-957d-5ee3316ddeba]
└ @ Base loading.jl:1423


LoadError: importing MyTestTwo into Main conflicts with an existing identifier

In [ ]:
;tree .

In [24]:
write("test/runtests.jl",In[23])

242

It did not work from Jupyter but will work from the command line

* A `@testset` groups different tests together 
* Behind `@test` there has to be boolean, if it evaluates to `true` the test is passed, if it evaluates to `false` the test is failed
* Different OSes and hardware can sometimes lead to slightly different numerical results, it is important to account for that when writing test with some kind of relative tolerence
* An easy way to do this is by using `≈` instead of `==`

### Random Numbers and Tests
* If your functions use random numbers at some point, we can make the results reproducible nonetheless by using a specific random seed

In [25]:
rand()

0.13850537979311484

In [26]:
using Random

Random.seed!(1234)

TaskLocalRNG()

In [27]:
rand()

0.32597672886359486

In [28]:
Random.seed!(1234)

TaskLocalRNG()

In [29]:
rand()

0.32597672886359486

The default random number generator of Julia should work across different OSes the same when a random seed is set.

## Documentation 

Every package also should have a documentation. The Julia package structure also has a dedicated place for that and libraries that help building the documentation. The most common library for that is `Documenter.jl`

In [30]:
;tree .

.
├── LICENSE
├── Manifest.toml
├── Project.toml
├── README.md
├── docs
│   ├── Manifest.toml
│   ├── Project.toml
│   ├── make.jl
│   └── src
│       └── index.md
├── src
│   ├── MyTestTwo.jl
│   └── more_functions.jl
└── test
    ├── function_tests.jl
    └── runtests.jl

4 directories, 12 files


In [31]:
;less docs/src/index.md

```@meta
CurrentModule = MyTestTwo
```

# MyTestTwo

Documentation for [MyTestTwo](https://github.com/my-github-user-name/MyTestTwo.jl).

```@index
```

```@autodocs
Modules = [MyTestTwo]
```


In [32]:
;less docs/make.jl

using MyTestTwo
using Documenter

DocMeta.setdocmeta!(MyTestTwo, :DocTestSetup, :(using MyTestTwo); recursive=true)

makedocs(;
    modules=[MyTestTwo],
    authors="My Name <mail@adress.com>",
    repo="https://github.com/my-github-user-name/MyTestTwo.jl/blob/{commit}{path}#{line}",
    sitename="MyTestTwo.jl",
    format=Documenter.HTML(;
        prettyurls=get(ENV, "CI", "false") == "true",
        edit_link="main",
        assets=String[],
    ),
    pages=[
        "Home" => "index.md",
    ],
)


* With `Documenter` we can write the documentation using Markdown, similar to what we are doing here in Jupyter notebooks
* `Documenter` also scans all source files for docstrings that are written befor the function definitons 

* The `@autodocs` macro automatically collects all docstrings of all functions in our package. 
* The preset that we generated with `PkgTemplates` will already give as a first draft of a documenation when we build it (we have to do this outside of a Jupyter notebook)
* The documentation is saved in the `docs/build` folder
* If you have LaTeX installed, it is also possible to get a pdf using LaTeX

* Instead of `@autodocs` we can also directly choose ourselves which functions are listed with the `@docs` macro: 

In [33]:
## One of our functions 

Here, we just list this one function
    
```@docs
add_two 
```

LoadError: syntax: extra token "just" after end of expression

In [34]:
write("docs/src/functions.md",In[33])

89

We can also add more pages to the documentation by editing the `make.jl` file. We just have to add them to the `pages` keyword. 

In [35]:
using MyTestTwo
using Documenter

DocMeta.setdocmeta!(MyTestTwo, :DocTestSetup, :(using MyTestTwo); recursive=true)

makedocs(;
    modules=[MyTestTwo],
    authors="My Name <mail@adress.com>",
    repo="https://github.com/my-github-user-name/MyTestTwo.jl/blob/{commit}{path}#{line}",
    sitename="MyTestTwo.jl",
    format=Documenter.HTML(;
        prettyurls=get(ENV, "CI", "false") == "true",
        assets=String[],
    ),
    pages=[
        "Home" => "index.md",
        "Functions" => "functions.md"
    ],
)

LoadError: importing MyTestTwo into Main conflicts with an existing identifier

In [36]:
write("docs/make.jl",In[35])

517

To make the documention: 
* Open Julia and change to the `docs` folder
* Activate the `docs` environment
* Execture `include(make.jl)`

* The documentation will not fully work if you just open it in your browser unfortunately (in this case it does though). You will either need to run a local webserver (e.g. with `python3 -m http.server` and then opening `localhost:8000` in your browser) or upload it somewhere.

* In the next lectures we will look into how to automate these building processes, uploading the documentation and also running the tests with CI/CD

Further recourses on Documenter.jl you can find in its [documentation](https://juliadocs.github.io/Documenter.jl/stable/)